In [ ]:
"""
Mapping.py

Created by John Canty on 02-18-2017

This module is based on Peter Comb's original Python code for aligning two-color data

"""

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp

def makeLSQSpline( xl, yl, xr, yr, n):
    """ Creates fitting function for mapping the left channel onto the right channel """
    
    xmin = min(xl)
    xmax = max(xl)
    ymin = min(yl)
    ymax = max(yl)
    
    xknots = np.linspace(xmin, xmax, n)
    yknots = np.linspace(ymin, ymax, n)
    
    xspline = interp.LSQBivariateSpline(xl, yl, xr, xknots, yknots)
    yspline = interp.LSQBivariateSpline(xl, yl, yr, xknots, yknots)
    
    def mapping(xl, yl):
        xr = xspline.ev(xl,yl)
        yr = yspline.ev(xl,yl)
        return xr, yr
    
    mapping.xspline = xspline
    mapping.yspline = yspline
    mapping.maptype = "LSQspline"
    return mapping

def findpairs(coords,radius):
    '''Finds pairs of points in both channels
    
    Parameters:
    
    coords = numpy array of x and y coordinates from both
    the left and right images
    
    radius = distance that is used as error metric for determining
    pairs of points. Radius is determined based on the spacing of the grid 
    generated experimentally. For grid spacing of 0.5 um we will use an 
    error radius of 50 nm
    '''
    pairs = []
    
    coordsl = coords[:,0:1]
    coordsr = coords[:,2:3]
    
    # Calculate pairwise distance matrix between points of both channels
    for i in range(coordsl.shape[0]):
        for j in range(coordsr.shape[0]):
            xl = coordsl[:,0][i]
            yl = coordsl[:,1][i]
            xr = coordsr[:,0][j]
            yr = coordsr[:,1][j]
            
            # If distance within error, add to pair
            dist = sqrt(abs(xl - xr)**2 + abs(yl - yr)**2)
            if dist < radius:
                pairs = np.append(pairs, [xl, yl, xr, yr], axis=0)

    return pairs

if __name__ == '__main__':
    main()
    
    


In [ ]:
import sys
import argparse

from Mapping import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import xlrd

def main():
    '''Main function docstring'''
    from scipy.io import loadmat
    
    # Load coordinates
    data = loadmat(matfile,struct_as_record = False, squeeze_me = True)
    coords = np.array(data)

    # Find pairs of points and sort
    pairs = findpairs(coords)
    xl = pairs[:,0]
    yl = pairs[:,1]
    xr = pairs[:,0]
    yr = pairs[:,1]
    
    # Perform mapping 
    reg_dev = []
    for i in range(0, pairs.shape[0]):
        # Save a point in the grid in both channels
        xl_p = xl[i]
        yl_p = yl[i]
        xr_p = xr[i]
        yr_p = xr[i]
                   
        # Save the rest of the points
        xl_n = xl[:i] + xl[i+1:]
        yl_n = yl[:i] + yl[i+1:]
        xr_n = xr[:i] + xr[i+1:]
        yr_n = yr[:i] + yr[i+1:]

        # Generate the mapping           
        mapping = makeLSQSpline( xl_n, yl_n, xr_n, yr_n, n)
        xr_map, yr_map = mapping(xl_p, yl_p)
        
        # Calculate target registration error
        RMSD = sqrt(abs(xr_map - xr_p)**2 + abs(yr_map - yr_p)**2)                    
        reg_dev = reg_dev.append(RMSD)
                
    mu = mean(reg_dev)
    sigma = std(reg_dev)
    
    # Plot original channels
    plt.plot(xr, yr, 'bo')
    plt.plot(xl, yl, 'ro')
    plt.show()
    
    # Plot registration error histogram
    plt.hist(reg_dev,bins = 15)
    plt.show()
                   
if __name__ == '__main__':
    main()